<h1>Getting Started</h1>
1. signup for Rasgo (https://app.rasgoml.com/account/register) and get your API key from the top right corner in the UI<br>
2. install pyrasgo (next cell down)<br>
3. check out the docs: https://docs.rasgoml.com/rasgo-docs/pyrasgo/user-defined-transforms-udts<br>
4. follow along with this tutorial

In [3]:
!pip install pyrasgo --upgrade
import pyrasgo

In [20]:
#use your rasgo API key to connect
api_key = ''
rasgo = pyrasgo.connect(api_key)

In [21]:
# Get all available transforms
# Best place to see more info is https://docs.rasgoml.com/rasgo-docs/pyrasgo/user-defined-transforms-udts
transforms = rasgo.get.transforms()
for t in transforms:
    args = []
    for a in t.arguments:
        args.append(a['name'])
    print(t.id, t.name, '({})'.format(', '.join(args)))

139 rasgo_unpivot (value_column, name_column, column_list_vals)
126 rasgo_math (math_ops)
144 rasgo_todate (date_columns, format_expression)
137 rasgo_pivot (dimensions, pivot_column, value_column, agg_method, list_of_vals)
124 rasgo_one_hot_encode (column)
128 rasgo_levenshtein (columns1, column2)
136 rasgo_union (source_id, union_all)
134 rasgo_filter (filter_statements)
138 rasgo_group_by (group_items, aggregations)
135 rasgo_datespine (date_col, start_timestamp, end_timestamp, interval_type)
125 rasgo_impute (imputations, flag_missing_vals)
143 rasgo_train_test_split (order_by, train_percent)
129 rasgo_movingavg (input_columns, window_sizes, order_by, partition)
142 rasgo_datetrunc (date_columns, date_part)
127 rasgo_lag (columns, amounts, partition, order_by)
130 rasgo_binning (binning_type, bucket_count, col_to_bucket)
141 rasgo_datepart (date_columns, date_part)


In [13]:
sources = rasgo.get.data_sources()
for source in sources:
    if 'ADVENTUREWORKS' in source.name:
        print(source.id, source.table)

2614 DIMACCOUNT
2615 DIMCURRENCY
2622 DIMSALESREASON
2616 DIMCUSTOMER
2621 DIMPROMOTION
2617 DIMDATE
2632 DIMPRODUCTCATEGORY
2618 DIMDEPARTMENTGROUP
2626 NEWFACTCURRENCYRATE
2619 DIMGEOGRAPHY
2620 DIMORGANIZATION
2623 DIMRESELLER
2624 FACTADDITIONALINTERNATIONALPRODUCTDESCRIPTION
2625 FACTPRODUCTINVENTORY
2627 FACTCALLCENTER
2629 FACTRESELLERSALES
2628 FACTINTERNETSALESREASON
2630 FACTSALESQUOTA
2631 PROSPECTIVEBUYER
2633 DIMPRODUCTSUBCATEGORY
2634 DIMSCENARIO
2635 FACTCURRENCYRATE
2636 FACTFINANCE
2637 FACTINTERNETSALES


In [16]:
# get some rasgo data sources to operate on
internet_sales = rasgo.get.data_source(id=2637)
customer = rasgo.get.data_source(id=2616)
dim_date = rasgo.get.data_source(id=2617)

<h2>Example 1: Forecast Direct Sales</h2>

In [35]:
# preview data set by generating a df and then previewing it with .head()
rasgo.read.source_data(internet_sales.id, limit=100).head()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05


In [46]:
# convert date string to date

t1 = internet_sales.transform(
  transform_name='todate',
  format_expression = 'YYYYMMDD',
  date_columns = ['ORDERDATEKEY'])

print(t1.preview_sql())

SELECT *,
DATE(ORDERDATEKEY, 'YYYYMMDD') as ORDERDATEKEY_todate 
from ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES


In [47]:
t1.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,ORDERDATEKEY_TODATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,1898.0944,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07,2010-12-31


In [37]:
# truncate date to week in a new column

t2 = t1.transform(
  transform_name='rasgo_datetrunc',
  date_part = 'week',
  date_columns = ['ORDERDATEKEY_TODATE'])

t2.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,ORDERDATEKEY_TODATE,ORDERDATEKEY_TODATE_WEEK
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29,2010-12-26
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29,2010-12-26
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29,2010-12-26
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29,2010-12-26
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05,2010-12-29,2010-12-26
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30,2010-12-26
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30,2010-12-26
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30,2010-12-26
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06,2010-12-30,2010-12-26
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07,2010-12-31,2010-12-26


In [38]:
# aggregate sales to week

t3 = t2.transform(
  transform_name='rasgo_group_by',
  group_items=['ORDERDATEKEY_TODATE_WEEK', 'PRODUCTKEY'],
  aggregations={
      'SALESAMOUNT': ['SUM', 'AVG'],
      'TAXAMT': ['SUM'],
      'FREIGHT': ['SUM']
  }
)

t3.preview()

,ORDERDATEKEY_TODATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM
0,2010-12-26,310,7156.5400,3578.2700000000,572.5232,178.9136
1,2010-12-26,346,10199.9700,3399.9900000000,815.9976,254.9994
2,2010-12-26,336,699.0982,699.0982000000,55.9279,17.4775
3,2010-12-26,311,7156.5400,3578.2700000000,572.5232,178.9136
4,2010-12-26,351,3374.9900,3374.9900000000,269.9992,84.3748
5,2010-12-26,312,7156.5400,3578.2700000000,572.5232,178.9136
6,2010-12-26,330,699.0982,699.0982000000,55.9279,17.4775
7,2010-12-26,313,3578.2700,3578.2700000000,286.2616,89.4568
8,2011-01-02,314,7156.5400,3578.2700000000,572.5232,178.9136
9,2011-01-02,336,699.0982,699.0982000000,55.9279,17.4775


In [39]:
# add lags for forecast model

t4 = t3.transform(
    transform_name='rasgo_lag',
    columns = ['SALESAMOUNT_SUM'],
    amounts = [1,2,3,4,26,52],
    order_by = ['ORDERDATEKEY_TODATE_WEEK'],
    partition = ['PRODUCTKEY']
)

t4.preview()

,ORDERDATEKEY_TODATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM,LAG_SALESAMOUNT_SUM_1,LAG_SALESAMOUNT_SUM_2,LAG_SALESAMOUNT_SUM_3,LAG_SALESAMOUNT_SUM_4,LAG_SALESAMOUNT_SUM_26,LAG_SALESAMOUNT_SUM_52
0,2012-12-23,530,4.9900,4.9900000000,0.3992,0.1248,None,None,None,None,None,None
1,2012-12-30,530,4.9900,4.9900000000,0.3992,0.1248,4.9900,None,None,None,None,None
2,2013-01-06,530,19.9600,4.9900000000,1.5968,0.4992,4.9900,4.9900,None,None,None,None
3,2013-01-13,530,4.9900,4.9900000000,0.3992,0.1248,19.9600,4.9900,4.9900,None,None,None
4,2013-01-20,530,4.9900,4.9900000000,0.3992,0.1248,4.9900,19.9600,4.9900,4.9900,None,None
5,2013-01-27,530,89.8200,4.9900000000,7.1856,2.2464,4.9900,4.9900,19.9600,4.9900,None,None
6,2013-02-03,530,134.7300,4.9900000000,10.7784,3.3696,89.8200,4.9900,4.9900,19.9600,None,None
7,2013-02-10,530,99.8000,4.9900000000,7.9840,2.4960,134.7300,89.8200,4.9900,4.9900,None,None
8,2013-02-17,530,104.7900,4.9900000000,8.3832,2.6208,99.8000,134.7300,89.8200,4.9900,None,None
9,2013-02-24,530,119.7600,4.9900000000,9.5808,2.9952,104.7900,99.8000,134.7300,89.8200,None,None


In [40]:
# apply 80/20 train-test split

t5 = t4.transform(
    transform_name='rasgo_train_test_split',
    order_by = ['ORDERDATEKEY_TODATE_WEEK'],
    train_percent = .8
)

t5.preview()

,ORDERDATEKEY_TODATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM,LAG_SALESAMOUNT_SUM_1,LAG_SALESAMOUNT_SUM_2,LAG_SALESAMOUNT_SUM_3,LAG_SALESAMOUNT_SUM_4,LAG_SALESAMOUNT_SUM_26,LAG_SALESAMOUNT_SUM_52,TT_SPLIT
0,2010-12-26,346,10199.9700,3399.9900000000,815.9976,254.9994,None,None,None,None,None,None,Train
1,2010-12-26,311,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
2,2010-12-26,344,3399.9900,3399.9900000000,271.9992,84.9998,None,None,None,None,None,None,Train
3,2010-12-26,314,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
4,2010-12-26,310,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
5,2010-12-26,336,699.0982,699.0982000000,55.9279,17.4775,None,None,None,None,None,None,Train
6,2010-12-26,330,699.0982,699.0982000000,55.9279,17.4775,None,None,None,None,None,None,Train
7,2010-12-26,351,3374.9900,3374.9900000000,269.9992,84.3748,None,None,None,None,None,None,Train
8,2010-12-26,312,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
9,2010-12-26,313,3578.2700,3578.2700000000,286.2616,89.4568,None,None,None,None,None,None,Train


In [43]:
# publish the new data set back to Rasgo
new_source = t5.to_source(new_table_name='WeeklyProductSalesLagged')
print(new_source)

Source(id=2645, name=RASGO.PUBLIC.WEEKLYPRODUCTSALESLAGGED, sourceType=table, table=RASGO.PUBLIC.WeeklyProductSalesLagged)


In [45]:
#interact with the published data set
weeklysales_df = rasgo.read.source_data(new_source.id)
weeklysales_df

,ORDERDATEKEY_TODATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM,LAG_SALESAMOUNT_SUM_1,LAG_SALESAMOUNT_SUM_2,LAG_SALESAMOUNT_SUM_3,LAG_SALESAMOUNT_SUM_4,LAG_SALESAMOUNT_SUM_26,LAG_SALESAMOUNT_SUM_52,TT_SPLIT
0,2010-12-26,346,10199.9700,3399.9900000000,815.9976,254.9994,None,None,None,None,None,None,Train
1,2010-12-26,311,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
2,2010-12-26,344,3399.9900,3399.9900000000,271.9992,84.9998,None,None,None,None,None,None,Train
3,2010-12-26,314,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
4,2010-12-26,310,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6930,2014-01-26,214,139.9600,34.9900000000,11.1968,3.4992,664.8100,839.7600,594.8300,559.8400,979.7200,1084.6900,Test
6931,2014-01-26,477,64.8700,4.9900000000,5.1896,1.6224,194.6100,219.5600,144.7100,214.5700,464.0700,279.4400,Test
6932,2014-01-26,489,215.9600,53.9900000000,17.2768,5.3992,323.9400,323.9400,215.9600,53.9900,431.9200,377.9300,Test
6933,2014-01-26,536,119.9600,29.9900000000,9.5968,2.9992,599.8000,479.8400,629.7900,509.8300,659.7800,299.9000,Test


<h1>COMING SOON</h1>
<h2>Project 2: Analyze Customer Behavior</h2>